# Lab 5-1: Blackjack with Monte Carlo Methods

**IE 7295 Reinforcement Learning | Sutton & Barto Chapter 5**

---

## Background

Monte Carlo methods learn directly from episodes of experience without requiring a model of the environment's dynamics. This lab implements the **First-Visit Monte Carlo** algorithm on the classic Blackjack problem from Sutton & Barto (2018), Example 5.1.

### Learning Objectives
- Understand Monte Carlo prediction methods
- Implement First-Visit MC algorithm
- Learn from sampled episodes of experience
- Estimate action-value functions Q(s,a)
- Visualize value functions and policies

### Blackjack Rules
- **Goal:** Get sum close to 21 without exceeding
- **Actions:** Hit (draw card) or Stick (stop)
- **States:** (player_sum, dealer_card, usable_ace)
- **Rewards:** +1 (win), 0 (draw), -1 (lose)
- **Ace:** Can be 1 or 11 (usable if 11)

---
## Section 1: Environment Setup and Dependencies

Importing libraries and configuring the computational environment

In [ ]:
# Cell 1: Import Libraries
import sys
import gymnasium as gym
import numpy as np
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")
print(f"Gymnasium version: {gym.__version__}")

---
## Section 2: Creating the Blackjack Environment

Initializing the OpenAI Gymnasium Blackjack-v1 environment

In [ ]:
# Cell 2: Initialize Blackjack Environment (v1)

# FIXED: Using v1 instead of v0
env = gym.make('Blackjack-v1')

print(f"Environment: Blackjack-v1")
print(f"Action space: {env.action_space}")
print(f"Number of actions: {env.action_space.n}")
print("Actions: 0 = Stick (stop), 1 = Hit (draw card)")

# Demonstrate environment
sample_state, _ = env.reset()
print(f"\nSample initial state: {sample_state}")
print(f"  Player sum: {sample_state[0]}")
print(f"  Dealer showing: {sample_state[1]}")
print(f"  Usable ace: {sample_state[2]}")

---
## Section 3: Monte Carlo ES Algorithm Overview

### Monte Carlo with Exploring Starts (MC ES)

Algorithm for finding optimal policy π ≈ π*

---

### Algorithm Overview

Monte Carlo ES uses **Exploring Starts** to ensure all state-action pairs are visited. Each episode begins with a **random** state-action pair, then follows the current policy. This guarantees exploration while still finding the optimal policy.

### Pseudocode

![Monte Carlo ES Pseudocode](https://github.com/mdehghani86/RL_labs/blob/master/Lab%2005/MCM_ES.jpg?raw=true)

*Figure: Monte Carlo ES Algorithm from Sutton & Barto*

---

### Key Steps

1. **Exploring Start:** Random (S₀, A₀) pair
2. **Generate Episode:** Follow current policy π
3. **Calculate Returns:** G for each (s,a)
4. **Update Q:** Average returns for each pair
5. **Policy Improvement:** π(s) ← argmax Q(s,a)

### Why Exploring Starts?

Without exploring starts, a deterministic policy might never visit certain state-action pairs, preventing us from learning their true values.

**Solution:** Start each episode with a random (s,a) pair to ensure comprehensive exploration of the state-action space.

---
## Section 4: Stochastic Policy for Exploration

Defining an arbitrary exploration policy to generate learning episodes

In [ ]:
# Cell 3: Episode Generation with Arbitrary Stochastic Policy

# ============================================================================
# IMPORTANT CONCEPT - ARBITRARY vs OPTIMAL POLICY
# ============================================================================
#
# 1. ARBITRARY POLICY (used during learning):
#    - A simple, reasonable policy we start with
#    - NOT optimal, but provides exploration
#    - In this code: threshold-based stochastic policy
#      * If player_sum > 18: 80% stick, 20% hit
#      * If player_sum <= 18: 20% stick, 80% hit
#    - Purpose: Generate episodes to learn Q-values
#
# 2. OPTIMAL POLICY (learned from Q-values):
#    - The BEST policy derived after learning
#    - Greedy with respect to learned Q-values
#    - π*(s) = argmax_a Q(s,a)
#    - This is what we are trying to find!
#
# LEARNING PROCESS:
#   Arbitrary Policy → Generate Episodes → Learn Q-values → Extract Optimal Policy
#
# ============================================================================

def play_episode_arbitrary_policy(env):
    """
    Play complete episode using ARBITRARY stochastic policy.
    This is NOT the optimal policy - it is our exploration policy.
    
    Returns:
        episode: List of (state, action, reward) tuples
    """
    episode = []
    state, _ = env.reset()
    
    while True:
        # ARBITRARY POLICY DEFINITION:
        # Simple threshold-based probabilities for exploration
        if state[0] > 18:
            # High sum: prefer to stick (conservative)
            action_probs = [0.8, 0.2]  # [P(stick), P(hit)]
        else:
            # Low sum: prefer to hit (aggressive)
            action_probs = [0.2, 0.8]
        
        # Sample action from arbitrary policy
        action = np.random.choice([0, 1], p=action_probs)
        
        # Execute action
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
        episode.append((state, action, reward))
        state = next_state
        
        if done:
            break
    
    return episode

# Test episode generation
print("Testing arbitrary policy episode generation...\n")
sample_episode = play_episode_arbitrary_policy(env)
print(f"Episode length: {len(sample_episode)} steps")
print(f"Final reward: {sample_episode[-1][2]}")
print(f"\nFirst 3 steps:")
for i, (state, action, reward) in enumerate(sample_episode[:3]):
    action_name = "Stick" if action == 0 else "Hit"
    print(f"  Step {i+1}: State={state}, Action={action_name}, Reward={reward}")

---
## Section 5: First-Visit Monte Carlo Q-Value Updates

Implementing the core learning algorithm for action-value estimation

In [ ]:
# Cell 4: Q-Value Update Function

def update_Q(episode, Q, returns_sum, N, gamma=1.0):
    """
    Update Q-values using First-Visit Monte Carlo.
    
    For each (state, action) pair in episode:
      1. Check if this is the FIRST visit to this pair
      2. Calculate return G from this point forward
      3. Update running average: Q(s,a) = mean(all returns)
    """
    visited = set()
    
    for t, (state, action, reward) in enumerate(episode):
        sa_pair = (state, action)
        
        # First-visit check
        if sa_pair not in visited:
            visited.add(sa_pair)
            
            # Calculate return G from time t
            G = sum((gamma ** k) * r for k, (_, _, r) in enumerate(episode[t:]))
            
            # Update statistics
            returns_sum[state][action] += G
            N[state][action] += 1.0
            
            # Update Q as running average
            Q[state][action] = returns_sum[state][action] / N[state][action]

print("Q-value update function ready")

In [ ]:
# Cell 5: Monte Carlo Prediction Loop

def mc_predict(env, num_episodes, gamma=1.0):
    """
    Monte Carlo prediction for Q-value estimation.
    Uses arbitrary policy for exploration.
    """
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    print(f"Starting Monte Carlo prediction with {num_episodes:,} episodes...\n")
    
    for i_episode in range(1, num_episodes + 1):
        episode = play_episode_arbitrary_policy(env)
        update_Q(episode, Q, returns_sum, N, gamma)
        
        if i_episode % 50000 == 0:
            print(f"Episode {i_episode:,}/{num_episodes:,}")
    
    print("\nMonte Carlo prediction complete")
    return Q

print("MC prediction function ready")

---
## Section 6: Visualization Functions

Creating 3D value function plots and 2D policy heatmaps

In [ ]:
# Cell 6: FIXED Visualization Functions
#
# FIXES:
# 1. Proper 3D surface plot configuration
# 2. Corrected 2D policy heatmap with proper axis alignment
# 3. Better colormap handling
# 4. Fixed coordinate systems

def plot_blackjack_values(V):
    """
    Create properly formatted 3D value function plots.
    
    This function creates two 3D surface plots showing state values:
    - One for states with usable ace
    - One for states without usable ace
    """
    def get_Z(player_sum, dealer_card, usable_ace):
        """
        Helper function to get value for a specific state.
        
        Parameters:
        - player_sum: Player's current card sum (12-21)
        - dealer_card: Dealer's visible card (1-10, where 1=Ace)
        - usable_ace: Boolean for whether player has usable ace
        
        Returns: State value V(s) if exists, else 0
        """
        state = (player_sum, dealer_card, usable_ace)
        return V.get(state, 0)
    
    def create_surface(usable_ace, ax):
        """
        Create a 3D surface plot for a given usable_ace condition.
        
        KEY COMPONENTS:
        ---------------
        1. Meshgrid (X, Y): Creates a grid of all (player_sum, dealer_card) combinations
        2. Z array: State values at each grid point
        3. Surface plot: 3D visualization where height = value
        """
        # STEP 1: Define the state space dimensions
        # Player sum ranges from 12 to 21 (cannot play with sum < 12)
        player_range = np.arange(12, 22)  # [12, 13, 14, ..., 21]
        
        # Dealer showing ranges from Ace(1) to 10
        dealer_range = np.arange(1, 11)   # [1, 2, 3, ..., 10]
        
        # STEP 2: Create coordinate grid
        # X, Y are 2D arrays where each point represents a state coordinate
        # X[i,j] = player_sum, Y[i,j] = dealer_card for all (i,j) combinations
        X, Y = np.meshgrid(player_range, dealer_range)
        
        # STEP 3: Compute Z values (state values) for each coordinate
        # Z is a 2D array: Z[i,j] = V(player_range[j], dealer_range[i], usable_ace)
        # This creates the height of the surface at each point
        Z = np.array([[get_Z(x, y, usable_ace) 
                      for x in player_range]   # Iterate over player sums (columns)
                     for y in dealer_range])   # Iterate over dealer cards (rows)
        
        # IMPORTANT: Z shape is (len(dealer_range), len(player_range))
        # Each row corresponds to a dealer card, each column to a player sum
        
        # STEP 4: Create the 3D surface plot
        surf = ax.plot_surface(
            X, Y, Z,                    # Coordinates and heights
            cmap=cm.coolwarm,           # Color: blue (low) to red (high)
            linewidth=0,                # No grid lines on surface
            antialiased=True,           # Smooth rendering
            vmin=-1, vmax=1,            # Value range for color mapping
            alpha=0.8                   # Slight transparency
        )
        
        # STEP 5: Label axes and set view angle
        ax.set_xlabel('Player Sum', fontsize=11)
        ax.set_ylabel('Dealer Showing', fontsize=11)
        ax.set_zlabel('Value', fontsize=11)
        ax.set_zlim(-1, 1)              # Z-axis limits
        ax.view_init(elev=25, azim=-130) # Viewing angle (elevation, azimuth)
        
        return surf
    
    # Create figure
    fig = plt.figure(figsize=(14, 11))
    
    # With usable ace
    ax1 = fig.add_subplot(211, projection='3d')
    ax1.set_title('State Values WITH Usable Ace', fontsize=13, fontweight='bold', pad=15)
    surf1 = create_surface(True, ax1)
    fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=10)
    
    # Without usable ace
    ax2 = fig.add_subplot(212, projection='3d')
    ax2.set_title('State Values WITHOUT Usable Ace', fontsize=13, fontweight='bold', pad=15)
    surf2 = create_surface(False, ax2)
    fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=10)
    
    plt.tight_layout()
    plt.show()


def plot_policy(policy):
    """
    Create properly formatted 2D policy heatmaps.
    
    Creates two heatmaps showing the optimal action for each state:
    - One for states with usable ace
    - One for states without usable ace
    
    Colors: Green = STICK (0), Red = HIT (1)
    """
    def get_action(player_sum, dealer_card, usable_ace):
        """
        Get the policy action for a specific state.
        
        Returns: 0 (STICK) or 1 (HIT), defaults to 1 if state not in policy
        """
        state = (player_sum, dealer_card, usable_ace)
        return policy.get(state, 1)  # Default: hit if state unseen
    
    def create_heatmap(usable_ace, ax):
        """
        Create a 2D heatmap showing policy actions.
        
        KEY COMPONENTS:
        ---------------
        1. Policy grid (Z): 2D array where Z[i,j] = action for that state
        2. Heatmap visualization: Color-coded policy decisions
        3. Axes: Player sum (x-axis), Dealer card (y-axis)
        """
        # STEP 1: Define state space ranges
        player_range = range(12, 22)  # Player sum: 12-21
        dealer_range = range(1, 11)   # Dealer card: 1-10 (Ace to 10)
        
        # STEP 2: Build the policy grid
        # Z is a 2D array where each element is the action (0=STICK, 1=HIT)
        # Rows represent dealer cards, columns represent player sums
        Z = np.array([[get_action(player, dealer, usable_ace)
                      for player in player_range]    # Columns: player sums
                     for dealer in dealer_range])    # Rows: dealer cards
        
        # IMPORTANT: Z[i,j] = policy action when:
        #   - Dealer shows dealer_range[i]
        #   - Player has sum player_range[j]
        # Z shape: (10 dealer cards, 10 player sums)
        
        # STEP 3: Create heatmap visualization
        im = ax.imshow(
            Z,
            cmap='RdYlGn_r',        # Red=HIT(1), Green=STICK(0)
            aspect='auto',          # Automatic aspect ratio
            vmin=0, vmax=1,         # Action values: 0 or 1
            extent=[11.5, 21.5, 0.5, 10.5],  # [x_min, x_max, y_min, y_max]
            origin='lower',         # Y-axis starts from bottom
            interpolation='nearest' # Sharp boundaries between actions
        )
        
        # EXPLANATION OF extent parameter:
        # extent=[11.5, 21.5, 0.5, 10.5] maps the array indices to coordinate values
        # - X (player sum): from 11.5 to 21.5 (centers values 12-21)
        # - Y (dealer card): from 0.5 to 10.5 (centers values 1-10)
        
        # STEP 4: Configure axis ticks and labels
        ax.set_xticks(range(12, 22))           # X-axis: 12, 13, ..., 21
        ax.set_yticks(range(1, 11))            # Y-axis: 1, 2, ..., 10
        ax.set_yticklabels(['A'] + list(range(2, 11)))  # Display A for Ace (value 1)
        
        ax.set_xlabel('Player Sum', fontsize=11)
        ax.set_ylabel('Dealer Showing', fontsize=11)
        
        # STEP 5: Add grid for readability
        ax.grid(True, color='black', linewidth=0.5, alpha=0.3)
        ax.set_axisbelow(False)  # Grid appears on top of heatmap
        
        # STEP 6: Add colorbar legend
        cbar = plt.colorbar(im, ax=ax, ticks=[0, 1], fraction=0.046, pad=0.04)
        cbar.ax.set_yticklabels(['STICK', 'HIT'])  # Label the actions
        
        return im
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # With usable ace
    ax1.set_title('Policy WITH Usable Ace', fontsize=12, fontweight='bold')
    create_heatmap(True, ax1)
    
    # Without usable ace
    ax2.set_title('Policy WITHOUT Usable Ace', fontsize=12, fontweight='bold')
    create_heatmap(False, ax2)
    
    plt.tight_layout()
    plt.show()

print("Visualization functions ready")

---
## Section 7: Running Monte Carlo Experiments

Executing large-scale learning and extracting the optimal policy

In [ ]:
# Cell 7: Execute Monte Carlo Learning
#
# ============================================================================
# CRITICAL EXPLANATION - TWO POLICIES:
# ============================================================================
#
# In this cell, we work with TWO different policies:
#
# 1. ARBITRARY POLICY (for learning):
#    --------------------------------
#    - The stochastic policy we use to GENERATE episodes
#    - Defined in play_episode_arbitrary_policy()
#    - Threshold-based: prefer stick if sum>18, hit if sum<=18
#    - Purpose: Explore the environment to learn Q-values
#    - This is NOT what we are trying to find!
#
# 2. OPTIMAL POLICY (what we are finding):
#    ------------------------------------
#    - The GREEDY policy extracted from learned Q-values
#    - Defined as: π*(s) = argmax_a Q(s,a)
#    - Deterministic: always picks best action
#    - This is the GOAL of our learning!
#
# PROCESS FLOW:
#   Arbitrary Policy → Episodes → Q-values → Optimal Policy
#   (exploration)     (data)    (learning)  (solution)
#
# ANALOGY:
#   - Arbitrary policy = practice games with exploration
#   - Q-values = knowledge learned from practice
#   - Optimal policy = tournament strategy using learned knowledge
#
# ============================================================================

# Run Monte Carlo prediction
NUM_EPISODES = 500000

print("="*60)
print("LEARNING PHASE: Using ARBITRARY policy for exploration")
print("="*60)
print(f"Episodes: {NUM_EPISODES:,}")
print("Arbitrary policy: Threshold-based stochastic")
print("Goal: Learn Q(s,a) values\n")

# Learn Q-values using arbitrary policy
Q = mc_predict(env, NUM_EPISODES)

print("\n" + "="*60)
print("EXTRACTION PHASE: Deriving OPTIMAL policy from Q-values")
print("="*60)

# Convert Q-values to state values under arbitrary policy
# V(s) = Σ π(a|s) * Q(s,a) for arbitrary policy
V_arbitrary = {}
for state, action_values in Q.items():
    if state[0] > 18:
        # Arbitrary policy: 80% stick, 20% hit
        V_arbitrary[state] = 0.8 * action_values[0] + 0.2 * action_values[1]
    else:
        # Arbitrary policy: 20% stick, 80% hit
        V_arbitrary[state] = 0.2 * action_values[0] + 0.8 * action_values[1]

# Extract OPTIMAL policy (greedy w.r.t. Q)
# This is the policy we are trying to find!
optimal_policy = {}
for state, action_values in Q.items():
    # Select action with highest Q-value (greedy)
    optimal_policy[state] = np.argmax(action_values)
    
print("\nOptimal policy extracted via greedy selection")
print("  π*(s) = argmax_a Q(s,a) for each state\n")

# Analysis
states_count = len(Q)
stick_count = sum(1 for a in optimal_policy.values() if a == 0)
hit_count = sum(1 for a in optimal_policy.values() if a == 1)

print("="*60)
print("RESULTS SUMMARY")
print("="*60)
print(f"States explored: {states_count}")
print(f"\nOptimal Policy Composition:")
print(f"  STICK states: {stick_count} ({100*stick_count/states_count:.1f}%)")
print(f"  HIT states:   {hit_count} ({100*hit_count/states_count:.1f}%)")
print(f"\nAverage state value: {np.mean(list(V_arbitrary.values())):.4f}")
print("\nInterpretation:")
print("  - Learned Q-values from arbitrary policy episodes")
print("  - Extracted optimal policy by always choosing best action")
print("  - Optimal policy is deterministic and greedy")
print("="*60)

In [ ]:
# Cell 8: Visualize Value Function

print("Generating 3D value function plots...\n")
plot_blackjack_values(V_arbitrary)

print("\nValue Function Interpretation:")
print("  • Red (high): Favorable states likely to win")
print("  • Blue (low): Unfavorable states likely to lose")
print("  • Peak near sum 20-21: Best positions")
print("  • Usable ace provides more flexibility")

In [ ]:
# Cell 9: Visualize OPTIMAL Policy

print("Generating optimal policy heatmaps...\n")
plot_policy(optimal_policy)

print("\nOptimal Policy Interpretation:")
print("  • Green = STICK (action 0)")
print("  • Red = HIT (action 1)")
print("  • Clear threshold around sum 17-20")
print("  • More aggressive hitting with usable ace")
print("  • Adapts to dealer's showing card")

---

## Key Findings

1. **Policy Learning:** We used an arbitrary exploration policy to generate episodes, then extracted the optimal policy from learned Q-values.

2. **Exploration vs Exploitation:** Arbitrary policy provides exploration, optimal policy is purely exploitative (greedy).

3. **Usable Ace Impact:** Optimal strategy differs significantly with usable ace - more aggressive hitting since cannot bust.

4. **Decision Boundaries:** Clear threshold emerges around sum 17-20 for stick/hit decision.

5. **Monte Carlo Strength:** Model-free learning directly from experience converges to optimal behavior.

---

## Questions for Reflection

1. Why do we need an exploration policy if we are trying to find the optimal policy?

2. What would happen if we used a purely greedy policy from the start?

3. How does First-Visit MC differ from Every-Visit MC?

4. Why is MC particularly suitable for Blackjack compared to Dynamic Programming?

5. How could we implement ε-greedy exploration instead of arbitrary policy?

---

**End of Lab 5-1: Blackjack with Monte Carlo Methods**